In [1]:
import pandas as pd
from bokeh.plotting import figure
from bokeh.io import show, output_notebook

** Data Exploration **

In [2]:
data = pd.read_csv('raw_data.csv')

In [3]:
data.head()

,order delivery time,order id,customer order rating,type of issue reported,region
0,2014-06-02 04:23:16 UTC,233599337,5,NaN,chi
1,2014-06-02 03:57:50 UTC,233599376,5,NaN,chi
2,2014-06-02 02:52:38 UTC,233599328,5,NaN,chi
3,2014-06-02 02:52:04 UTC,233599070,5,NaN,chi
4,2014-06-02 02:41:43 UTC,233599100,5,NaN,chi


In [4]:
# include 33 days of delivery data, 4 full weeks, 5 weekends
print('Minimum order delivery time: %s' %str(data['order delivery time'].min()))
print('Maximum order delivery time: %s' %str(data['order delivery time'].max()))

Minimum order delivery time: 2014-05-01 08:54:00 +0000
Maximum order delivery time: 2014-06-02 06:28:37 +0000


In [5]:
# Types of issue reported
print(data['type of issue reported'].dropna().unique())

['Wrong item' 'Item charged incorrectly' 'Damaged or spoiled'
 'Item missing' 'Other Order Issue' 'Poor service' 'Poor replacement']


In [6]:
from collections import Counter
Counter(data['type of issue reported'].fillna('Null'))

Counter({'Damaged or spoiled': 310,
         'Item charged incorrectly': 21,
         'Item missing': 178,
         'Null': 13870,
         'Other Order Issue': 21,
         'Poor replacement': 54,
         'Poor service': 129,
         'Wrong item': 374})

In [7]:
# check uniqueness of order id
'unique order id %d, total rows %d' %(len(data['order id'].unique()), len(data))

'unique order id 13845, total rows 14957'

In [8]:
# clean region type, set time zone by region
data['region'] = data['region'].str.strip()
data['region'].unique()
data['time zone'] = data['region'].replace({
    'nyc':'America/New_York',
    'sf': 'America/Los_Angeles',
    'chi': 'America/Chicago'
})

In [9]:
# change UTC to local timezone
from datetime import datetime
from dateutil.tz import *
def to_local_tz(curr_time, tz):
    utc_tz = gettz('UTC')
    local_tz = gettz(tz)
    utc_ts = curr_time.replace(tzinfo=utc_tz)
    local_ts = utc_ts.astimezone(local_tz)
    return local_ts

In [10]:
data['order delivery timestamp'] = pd.to_datetime(data['order delivery time'])

In [11]:
data['local delivery timestamp'] = data.apply(lambda row: to_local_tz(row['order delivery timestamp'], row['time zone']), axis=1)

In [12]:
data['local delivery date'] = data['local delivery timestamp'].astype(str).str.split().str[0]
data['local delivery time'] = data['local delivery timestamp'].astype(str).str.split().str[1]
data['local delivery time'] = data['local delivery time'].str.split('-').str[0]

In [13]:
data.to_csv('processed_data.csv', index=False)